In [ ]:

import os
import cdsapi
from typing import Optional, Dict, Any
from dotenv import load_dotenv

class CDSAPIConfig:
    """Configuration class for CDS API with multiple credential sources"""
    
    def __init__(self, url: Optional[str] = None, key: Optional[str] = None, env_file: Optional[str] = None):
        """
        Initialize CDS API configuration
        
        Args:
            url: CDS API URL (optional, will use environment variable or default)
            key: CDS API key (optional, will use environment variable)
            env_file: Path to .env file to load (optional)
        """
        # Load .env file if specified
        if env_file:
            load_dotenv(env_file)
        else:
            # Try to load .env file from current directory
            load_dotenv()
        
        self.url = url or os.getenv('CDSAPI_URL', 'https://cds.climate.copernicus.eu/api')
        self.key = key or os.getenv('CDSAPI_KEY')
        
        if not self.key:
            raise ValueError("CDS API key is required. Set CDSAPI_KEY environment variable or pass key parameter.")
    
    def get_client(self) -> cdsapi.Client:
        """Create and return a configured CDS API client"""
        return cdsapi.Client(url=self.url, key=self.key)

In [23]:
c = CDSAPIConfig(env_file='.env').get_client()
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'netcdf',
        'variable': ['total_precipitation', '2m_temperature'],
        'year': '2024',
        'month': '01',
        'day': list(range(1, 32)),
        'time': [f'{h:02d}:00' for h in range(0, 24, 3)],
        'area': [23.5, 102.0, 8.0, 110.0],  # Vietnam bounding box
        'product_type': 'reanalysis',
        'expver': '1',  # Add experiment version parameter
        'grid': [0.1, 0.1],  # Specify grid resolution
    },
    'era5_land_hourly_202401.nc'
)

2025-08-13 14:44:58,242 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-13 14:44:59,292 INFO Request ID is e8c57d19-0ef0-43fe-b9c7-b0e12bc210c6
2025-08-13 14:44:59,618 INFO status has been updated to accepted
2025-08-13 14:45:14,442 INFO status has been updated to running
2025-08-13 14:46:56,296 INFO status has been updated to successful


'era5_land_hourly_202401.nc'

In [27]:
import zipfile
import xarray as xr
import numpy as np
from era5_data_download import CDSAPIConfig
import shutil

# Step 1: Download data
print("Downloading ERA5-Land data...")
c = CDSAPIConfig(env_file='.env').get_client()
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'netcdf',
        'product_type': 'reanalysis',
        'variable': ['total_precipitation', '2m_temperature'],
        'year': '2024',
        'month': '01',
        'day': list(range(1, 32)),
        'time': [f'{h:02d}:00' for h in range(0, 24, 3)],
        'area': [23.5, 102.0, 8.0, 110.0],  # Vietnam
        'grid': [0.1, 0.1]
    },
    'era5_vietnam_raw.nc'
)

# Step 2: Auto-extract ZIP if needed
downloaded_file = 'era5_vietnam_raw.nc'
if zipfile.is_zipfile(downloaded_file):
    print("Extracting ZIP file...")
    with zipfile.ZipFile(downloaded_file, 'r') as zip_ref:
        nc_files = [f for f in zip_ref.namelist() if f.endswith('.nc')]
        zip_ref.extract(nc_files[0])
        shutil.move(nc_files[0], 'era5_vietnam_extracted.nc')
        netcdf_file = 'era5_vietnam_extracted.nc'
else:
    netcdf_file = downloaded_file

# # Step 3: Resample to 30m resolution
# print("🔍 Resampling to 30-meter resolution...")
# ds = xr.open_dataset(netcdf_file)

# # Calculate 30m grid
# meters_per_degree = 111319
# degree_resolution = 30 / meters_per_degree  # ~0.00027 degrees

# lat_min, lat_max = float(ds.latitude.min()), float(ds.latitude.max())
# lon_min, lon_max = float(ds.longitude.min()), float(ds.longitude.max())

# new_lats = np.arange(lat_min, lat_max, degree_resolution)
# new_lons = np.arange(lon_min, lon_max, degree_resolution)

# print(f"Original: {ds.latitude.size} x {ds.longitude.size}")
# print(f"30m grid: {len(new_lats)} x {len(new_lons)}")

# # Interpolate to 30m
# ds_30m = ds.interp(latitude=new_lats, longitude=new_lons, method='linear')

# # Save with compression
# ds_30m.to_netcdf('era5_vietnam_30m.nc', 
#                  encoding={var: {'zlib': True, 'complevel': 4} 
#                           for var in ds_30m.data_vars})

# print("Done! Final file: era5_vietnam_30m.nc")
# ds.close()
# ds_30m.close()

2025-08-13 15:12:26,489 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-13 15:12:27,165 INFO Request ID is 3d3aec98-6d5b-4383-a95a-28b20aa4b589
INFO:ecmwf.datastores.legacy_client:Request ID is 3d3aec98-6d5b-4383-a95a-28b20aa4b589
2025-08-13 15:12:27,479 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-08-13 15:12:36,873 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-08-13 15:13:45,246 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful
INFO:multiurl.base:Downloading https://object-store.os-api.cci2.ecmwf.int:443/cci2-prod-cache-3/2025-08-13/5c5d4a295e93

Extracting ZIP file...


In [29]:
import xarray as xr

ds = xr.open_dataset('era5_vietnam_extracted.nc')
# Global metadata
global_attrs = ds.attrs  # E.g., {'Conventions': 'CF-1.6', 'history': '...'}
# Variable metadata
var_attrs = {var: ds[var].attrs for var in ds.variables if var not in ds.coords}
# E.g., {'t2m': {'units': 'K', 'long_name': '2 metre temperature'}, ...}
# Coordinates and time
lats = ds.latitude.values
lons = ds.longitude.values
times = ds.valid_time.values

lats, lons, times

(array([23.5, 23.4, 23.3, 23.2, 23.1, 23. , 22.9, 22.8, 22.7, 22.6, 22.5,
        22.4, 22.3, 22.2, 22.1, 22. , 21.9, 21.8, 21.7, 21.6, 21.5, 21.4,
        21.3, 21.2, 21.1, 21. , 20.9, 20.8, 20.7, 20.6, 20.5, 20.4, 20.3,
        20.2, 20.1, 20. , 19.9, 19.8, 19.7, 19.6, 19.5, 19.4, 19.3, 19.2,
        19.1, 19. , 18.9, 18.8, 18.7, 18.6, 18.5, 18.4, 18.3, 18.2, 18.1,
        18. , 17.9, 17.8, 17.7, 17.6, 17.5, 17.4, 17.3, 17.2, 17.1, 17. ,
        16.9, 16.8, 16.7, 16.6, 16.5, 16.4, 16.3, 16.2, 16.1, 16. , 15.9,
        15.8, 15.7, 15.6, 15.5, 15.4, 15.3, 15.2, 15.1, 15. , 14.9, 14.8,
        14.7, 14.6, 14.5, 14.4, 14.3, 14.2, 14.1, 14. , 13.9, 13.8, 13.7,
        13.6, 13.5, 13.4, 13.3, 13.2, 13.1, 13. , 12.9, 12.8, 12.7, 12.6,
        12.5, 12.4, 12.3, 12.2, 12.1, 12. , 11.9, 11.8, 11.7, 11.6, 11.5,
        11.4, 11.3, 11.2, 11.1, 11. , 10.9, 10.8, 10.7, 10.6, 10.5, 10.4,
        10.3, 10.2, 10.1, 10. ,  9.9,  9.8,  9.7,  9.6,  9.5,  9.4,  9.3,
         9.2,  9.1,  9. ,  8.9,  8.8, 